In [4]:
import numpy as np
import cv2
import pandas as pd
from skimage.feature import local_binary_pattern, graycomatrix, graycoprops
from math import sqrt
import os
from tqdm import tqdm

In [46]:
def lbp_feature(image):
    if len(image.shape) > 2:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    i_max = np.max(image)
    i_min = np.min(image)
    if i_max - i_min != 0:
        image = (image - i_min) / (i_max - i_min)
    lbp = local_binary_pattern(image, R=1, P=8, method="uniform")  
    hist, bins = np.histogram(lbp.ravel(), bins=10, range=(0, 10))
    return {f"LBP_{i}": hist[i] for i in range(len(hist))}

In [32]:
def texture_feature(image):
    gray_img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    hist = cv2.calcHist([gray_img],[0],None, [256], [0,256])
    hist = hist/hist.sum()

    intensities = np.arange(256)
    mean = np.sum(intensities * hist.flatten())
    std = np.sqrt(np.sum((intensities-mean)**2 *hist.flatten()))
    uniformity = np.sum(hist.flatten()**2)
    third_moment = np.sum((intensities-mean)**3 * hist.flatten())
    return {
        "texture_mean": mean,
        "texture_std": std,
        "texture_uniformity": uniformity,
        "texture_third_moment": third_moment
    }

In [7]:
def color_feature(image):
    b, g, r = cv2.split(image)
    mean_r, mean_g, mean_b = np.mean(r), np.mean(g), np.mean(b)
    rs, gs, bs = sqrt(mean_r), np.sqrt(mean_g), np.sqrt(mean_b)
    return {"mean_r": mean_r,
        "mean_g": mean_g,
        "mean_b": mean_b,
        "red_sqr": rs,
        "green_sqr": gs,
        "blue_sqr": bs,
    }

In [8]:
def basic_feature(image):
    if len(image.shape) > 2:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(image, (3, 3), 0)
    threshold, new_img = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    
    area = np.count_nonzero(new_img)
    contours, _ = cv2.findContours(new_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contour = max(contours, key=cv2.contourArea)
    x, y, w, h = cv2.boundingRect(contour)
    
    length = x + w
    width = y + h
    ratio = length / width
    
    ellipse = cv2.fitEllipse(contour)
    
    major_axis_length = max(ellipse[1])
    minor_axis_length = min(ellipse[1])
    
    hull = cv2.convexHull(contour)
    hull_area = cv2.contourArea(hull)
    hull_perimeter = cv2.arcLength(hull, True)
    return {
        "area": area,
        "length": length,
        "width": width,
        "ratio": ratio,
        "major_axis_length": major_axis_length,
        "minor_axis_length": minor_axis_length,
        "convex_hull_area": hull_area,
        "convex_hull_perimeter": hull_perimeter,
    }

In [34]:
def compute_glcm_descriptor(image):
    if image is None:
        return None
    
    if len(image.shape) > 2:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    distance = [3]  
    angles = [0, np.pi/4, np.pi/2, 3*np.pi/4] 
    properties = ['contrast', 'correlation', 'energy', 'homogeneity']  
    
    glcm = graycomatrix(image, distances=distance, angles=angles, symmetric=True, normed=True)
    
    features = []
    for prop in properties:
        feature = graycoprops(glcm, prop).flatten()
        features.extend(feature)
    
    return np.array(features)

In [62]:
def create_gabor_filter(size, u0, v0, delta_x, delta_y):
    #size: kernel size
    #u0, v0: spatial frequency points
    #delta_x, delta_y: spatial scales

    y, x = np.mgrid[-size//2:size//2, -size//2:size//2]
    
    gaussian = np.exp(-0.5 * (x**2/delta_x**2 + y**2/delta_y**2))

    sinusoid = np.exp(-2j * np.pi * (u0*x + v0*y))
    
    return gaussian * sinusoid

def compute_gist_descriptor(image):

    if image is None:
        return None
    
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = image.astype(np.float32) / 255.0
    
    local_mean = cv2.GaussianBlur(image, (5, 5), 1.0)
    variance = cv2.GaussianBlur(image**2, (5, 5), 1.0) - local_mean**2
    variance[variance < 0] = 0 
    local_std = np.sqrt(variance)
    image = (image - local_mean) / (local_std + 1e-8)
    
    features = []
    scales = [2, 4, 8, 16]
    orientations = 8
    
    for delta in scales:
        for theta in range(orientations):
            angle = theta * np.pi / orientations
            u0 = np.cos(angle) / delta
            v0 = np.sin(angle) / delta
            
            gabor_filter = create_gabor_filter(size=31,u0=u0,v0=v0,delta_x=delta,delta_y=delta)
            
            filtered = cv2.filter2D(image, cv2.CV_32F, np.real(gabor_filter))
            
            block_h = image.shape[0] // 4
            block_w = image.shape[1] // 4
            
            for i in range(4):
                for j in range(4):
                    block = filtered[i*block_h:(i+1)*block_h, j*block_w:(j+1)*block_w]
                    energy = np.mean(np.abs(block))
                    features.append(energy)
    
    return np.array(features)


In [63]:
def extract_all_features(image):    
    features = {}

    lbp_hist = lbp_feature(image)
    features.update(lbp_hist) 
    
    texture_features = texture_feature(image)
    features.update(texture_features) 

    color_features = color_feature(image)
    features.update(color_features) 

    shape_features = basic_feature(image)
    if shape_features is not None:
        features.update(shape_features)

    glcm_features = compute_glcm_descriptor(image)
    if glcm_features is not None:
        for i, val in enumerate(glcm_features):
            features[f'GLCM_{i}'] = val

    gist_features = compute_gist_descriptor(image)
    if gist_features is not None:
        for i, val in enumerate(gist_features):
            features[f'GIST_{i}'] = val

    return features


def process_directory(base_path):
    all_data = []
    image_paths = []

    for root, dirs, files in os.walk(base_path):
        for file in files:
            if file.endswith(('.png', '.jpg', '.jpeg')):
                label = 0 if 'Negative' in root else 1
                image_paths.append((os.path.join(root, file), label))

    for image_path, label in tqdm(image_paths, desc="Processing Images"):
        image = cv2.imread(image_path)
        if image is None:
            continue  

        features = extract_all_features(image)
        features["Label"] = label
 
        all_data.append(features)

    df = pd.DataFrame(all_data)

    return df

In [65]:
base_path = r'/home/duyle/Rice_photos/BC-15'
df = process_directory(base_path)

Processing Images: 100%|██████████| 3677/3677 [05:02<00:00, 12.16it/s]


In [69]:
df

,LBP_0,LBP_1,LBP_2,LBP_3,LBP_4,LBP_5,LBP_6,LBP_7,LBP_8,LBP_9,...,GIST_503,GIST_504,GIST_505,GIST_506,GIST_507,GIST_508,GIST_509,GIST_510,GIST_511,Label
0,589,1164,791,2289,3315,3192,1773,1367,2066,2254,...,6.153780,4.732004,4.752348,4.196944,4.937119,4.817426,3.857685,3.996897,3.971284,0
1,657,1111,740,1787,2560,2484,1474,1307,1826,2254,...,5.306226,5.767577,4.192934,4.366243,4.047829,4.557763,3.889102,3.419347,3.457796,0
2,565,1006,704,1909,2955,2753,1480,1187,1685,2003,...,5.770615,5.161266,4.602048,5.123860,6.780639,4.031583,4.648014,4.179079,4.005245,0
3,594,1153,701,1932,2986,2592,1608,1402,2008,2224,...,5.524634,7.424312,5.007694,4.161709,4.208960,5.423459,5.040381,3.993508,3.662773,0
4,529,1127,736,2389,3981,3430,1634,1173,1768,2421,...,7.796167,6.548311,5.531692,4.785046,7.321709,5.458099,4.590240,4.637808,4.401915,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3672,566,1144,788,2213,3359,3035,1702,1344,1730,2663,...,5.679554,5.631991,4.199759,4.388726,5.635378,5.174569,3.686354,4.057350,3.794700,1
3673,480,1080,574,1830,2751,2724,1465,1150,1770,2440,...,6.243743,8.027259,5.875374,4.591694,6.247918,5.707466,4.321327,3.570124,4.989719,1
3674,319,854,547,2153,3365,3203,1485,1108,1796,1522,...,6.383222,4.864499,4.940446,4.717282,4.842475,5.623338,3.206896,3.966377,5.262681,1
3675,543,973,643,1762,2702,2478,1423,1130,1754,1904,...,6.578649,4.558363,4.895084,4.948411,5.048751,4.468331,4.526423,4.296866,4.038802,1


In [68]:
df.to_csv('dog.csv',index=False)